In [10]:
import os
import time
import logging
from functools import wraps
from flask import Flask, request, abort
from slack_bolt import App
from slack_bolt.adapter.flask import SlackRequestHandler
from slack_sdk.signature import SignatureVerifier
from slack_sdk.errors import SlackApiError
from dotenv import find_dotenv, load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from PyPDF2 import PdfReader

# Load environment variables
load_dotenv(find_dotenv())

# Set Slack API credentials
SLACK_BOT_TOKEN = os.environ["SLACK_BOT_TOKEN"]
SLACK_SIGNING_SECRET = os.environ["SLACK_SIGNING_SECRET"]

# Initialize Slack app
slack_app = App(token=SLACK_BOT_TOKEN)
signature_verifier = SignatureVerifier(SLACK_SIGNING_SECRET)

# Initialize Flask app
flask_app = Flask(__name__)
handler = SlackRequestHandler(slack_app)

# Slack verification decorator
def require_slack_verification(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        if not verify_slack_request():
            abort(403)
        return f(*args, **kwargs)
    return decorated_function

# Slack request verification function
def verify_slack_request():
    timestamp = request.headers.get("X-Slack-Request-Timestamp", "")
    signature = request.headers.get("X-Slack-Signature", "")

    current_timestamp = int(time.time())
    if abs(current_timestamp - int(timestamp)) > 60 * 5:
        return False

    return signature_verifier.is_valid(
        body=request.get_data().decode("utf-8"),
        timestamp=timestamp,
        signature=signature,
    )

# PDF extraction and analysis function
def extract_and_analyze_pdf(pdf_file_path):
    # Read and extract text from the PDF file
    reader = PdfReader(pdf_file_path)
    raw_text = ''
    for page in reader.pages:
        text = page.extract_text()
        if text:
            raw_text += text

    # Split the text into smaller chunks for processing
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    texts = text_splitter.split_text(raw_text)

    # Create embeddings and a FAISS vector store
    embeddings = OpenAIEmbeddings()
    docsearch = FAISS.from_texts(texts, embeddings)

    # Return the vector store for further processing
    return docsearch

# Handle Slack events for messages
@slack_app.event("message")
def handle_message_events(event, say):
    user_text = event.get("text", "").strip().lower()
    
    # Example greeting response
    if user_text in ["hi", "hello", "hey"]:
        say("Hello! I'm Alphie, your AI assistant. How can I assist you today?")
        return
    
    # PDF-related queries
    if "pdf" in user_text or "article" in user_text:
        docsearch = extract_and_analyze_pdf("Article 17.pdf")
        chain = load_qa_chain(OpenAI(), chain_type="stuff")

        query = user_text
        docs = docsearch.similarity_search(query)
        response = chain.run(input_documents=docs, question=query)
        
        say(response)
        return
    
    # Default response
    say("I didn't understand that. How can I help you today?")

# Slack events endpoint for Flask
@flask_app.route("/slack/events", methods=["POST"])
@require_slack_verification
def slack_events():
    return handler.handle(request)

# Run the Flask app
if __name__ with __name__ == "__main__":
    logging.info("Flask app started")
    flask_app.run(host="0.0.0.0", port=8000)


SyntaxError: invalid syntax (4225966052.py, line 115)